In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import time
from twilio.rest import Client
from datetime import datetime
import os
from keras import backend as K

# Twilio credentials
TWILIO_ACCOUNT_SID = ''  # Replace with your Twilio Account SID
TWILIO_AUTH_TOKEN = ''    # Replace with your Twilio Auth Token
TWILIO_PHONE_NUMBER = ''  # Replace with your Twilio phone number
EMERGENCY_NUMBER = ''  # Replace with your phone number (include country code)

def send_emergency_sms(confidence):
  """Send emergency SMS with detection confidence"""
  try:
      client = Client(TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN)
      message_body = f"⚠️ ALERT: Fight detected on webcam! Confidence: {confidence:.2%}"
      
      message = client.messages.create(
          body=message_body,
          from_=TWILIO_PHONE_NUMBER,
          to=EMERGENCY_NUMBER
      )
      print(f"Emergency SMS sent successfully")
      return True
  except Exception as e:
      print(f"Failed to send SMS: {str(e)}")
      return False

def process_frame_for_prediction(frame, target_size=(128, 128)):
  """Preprocess single frame for prediction"""
  resized_frame = cv2.resize(frame, target_size)
  normalized_frame = resized_frame.astype(np.float32) / 255.0
  return normalized_frame

def monitor_webcam(model_path, confidence_threshold=0.75, cooldown_period=60):
  """Monitor webcam feed for fight detection"""
  try:
      # Load model
      model = keras.models.load_model(model_path, 
                                    custom_objects={'recall_m': recall_m,
                                                  'precision_m': precision_m,
                                                  'f1_m': f1_m})
      print("Model loaded successfully")
  except Exception as e:
      print(f"Error loading model: {str(e)}")
      return

  # Create output directory for detected frames
  output_dir = 'detected_fights_webcam'
  os.makedirs(output_dir, exist_ok=True)

  # Initialize webcam
  cap = cv2.VideoCapture(0)
  if not cap.isOpened():
      print("Error: Could not open webcam")
      return

  frames_buffer = []
  last_alert_time = 0
  frame_count = 0

  print("Monitoring webcam... Press 'q' to quit")

  try:
      while True:
          ret, frame = cap.read()
          if not ret:
              print("Error reading from webcam")
              break

          # Process frame
          processed_frame = process_frame_for_prediction(frame)
          frames_buffer.append(processed_frame)

          # Maintain buffer size of 42 frames
          if len(frames_buffer) > 42:
              frames_buffer.pop(0)

          # Make prediction when we have enough frames
          if len(frames_buffer) == 42:
              current_time = time.time()
              
              # Prepare batch for prediction
              batch = np.expand_dims(np.array(frames_buffer), axis=0)
              
              # Make prediction
              prediction = model.predict(batch, verbose=0)
              predicted_class = np.argmax(prediction[0])
              confidence = prediction[0][predicted_class]

              # Display prediction info on frame
              text_color = (0, 255, 0)  # Green for normal
              status_text = "Status: Normal"
              
              if predicted_class == 1 and confidence >= confidence_threshold:
                  text_color = (0, 0, 255)  # Red for fight detected
                  status_text = f"FIGHT DETECTED! Conf: {confidence:.2%}"
                  
                  # Send alert if cooldown period has passed
                  if current_time - last_alert_time >= cooldown_period:
                      print(f"\nFight detected!")
                      print(f"Confidence: {confidence:.2%}")
                      
                      # Save frame
                      timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
                      output_path = os.path.join(output_dir, 
                                               f'fight_detected_{timestamp}.jpg')
                      cv2.imwrite(output_path, frame)
                      
                      # Send alert
                      if send_emergency_sms(confidence):
                          last_alert_time = current_time

              # Add text to frame
              cv2.putText(frame, status_text, (10, 30), 
                         cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)
              cv2.putText(frame, f"Press 'q' to quit", (10, 60), 
                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

          # Display frame
          cv2.imshow('Fight Detection System', frame)

          # Check for 'q' key to quit
          if cv2.waitKey(1) & 0xFF == ord('q'):
              print("Stopping monitoring...")
              break

  except KeyboardInterrupt:
      print("\nMonitoring interrupted by user")
  except Exception as e:
      print(f"\nError during monitoring: {str(e)}")
  finally:
      cap.release()
      cv2.destroyAllWindows()
      print("\nMonitoring stopped")

def main():
  print("Fight Detection System - Webcam Monitor")
  
  # Specify model path
  model_path = 'path_to_your_vivit_model'  # Replace with your model path
  
  # Set sensitivity parameters
  confidence_threshold = 0.75  # Minimum confidence to trigger alert
  cooldown_period = 60        # Minimum seconds between alerts
  
  # Start monitoring
  monitor_webcam(model_path, confidence_threshold, cooldown_period)

if __name__ == "__main__":
  main()